In [102]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk

from nltk.probability import *
from nltk.tokenize import wordpunct_tokenize
from collections import defaultdict
#------------------------------------get files from directory-------------------------------------
def getFilesFromDir (path):
    dir_content = os.listdir(path)
    dir_clean = filter(lambda x: (".DS_Store" not in x) and ("cmds" not in x), dir_content)
    msg = map(lambda x: getFullMessage(path + '/' + x), dir_clean)
    return msg
#------------------------------------get message from files---------------------------------------
#get full message 
def getFullMessage (path):
    fo = open (path)
    lines = fo.readlines()
    fo.close()
    lines = ''.join(lines)
    return lines
#get message subject
def getSubjectFromEmail (path):
    lines = getFilesFromDir(path)
    #subject index and subject end index
    subject = map(lambda x: x[x.index('Subject:')+8: len(x)], lines)
    subject = map(lambda x: x[: x.index('\n')], subject)
    subject = map(lambda x: x.lower(), subject)
    subject = map(lambda x: re.sub('re:', '', x), subject)
    return subject
#get message body
def getMessageBodyFromEmail (path):
    lines = getFilesFromDir(path)
    body = map(lambda x: x[x.index('\n\n'): len(x)], lines)
    body = map(lambda x: x.lower(), body)
    body = map(lambda x: re.sub('\n', ' ', x), body)
    body = map(lambda x: re.sub('[^a-zA-Z\s]', '', x), body)
    return body
#get sender
def getSenderFromEmail (path):
    lines = getFilesFromDir(path)
    sender = map(lambda x: x[x.index('From:'): len(x)], lines)
    sender = map(lambda x: x[: x.index('\n')], sender)
    for index in range(len(sender)):
        if '<' in sender[index]:
            sender[index] = sender[index][sender[index].index('<')+1: sender[index].index('>')]
        else:
            sender[index] = sender[index][sender[index].index('From:') + 5: ]
            if '(' in sender[index]:
                sender[index] = sender[index][: sender[index].index('(')]
        sender[index] = re.sub(' ', '', sender[index])
    return sender
#get date
def getDateFromEmail (path):
    lines = getFilesFromDir(path)
    date = map(lambda x: x[x.index('Date:')+5: len(x)], lines)
    date = map(lambda x: x[: x.index('\n')], date)
    return date
#get stop words
def getStopWords (path):
    fo = open (path)
    lines = fo.readlines()
    lines = map(lambda x: str.replace(x, '\n', ''), lines)
    fo.close()
    return lines
#------------------------------------parameter setting--------------------------------------------
#file path
base_path = "/Users/yuliang/Downloads/naive_bayes"
ham_path1 = base_path + "/easy_ham"
ham_path2 = base_path + "/easy_ham_2"
spam_path1 = base_path + "/spam"
spam_path2 = base_path + "/spam_2"
#------------------------------------get mail content and stop words------------------------------
hamFullTrain = getFilesFromDir(ham_path1)
hamSubjectTrain = getSubjectFromEmail(ham_path1)
hamBodyTrain = getMessageBodyFromEmail(ham_path1)
hamSenderTrain = getSenderFromEmail(ham_path1)
hamDateTrain = getDateFromEmail(ham_path1)

spamFullTrain = getFilesFromDir(spam_path1)
spamSubjectTrain = getSubjectFromEmail(spam_path1)
spamBodyTrain = getMessageBodyFromEmail(spam_path1)
spamSenderTrain = getSenderFromEmail(spam_path1)
spamDateTrain = getDateFromEmail(spam_path1)

stop_words_given = getStopWords ("/Users/yuliang/Downloads/naive_bayes/stopwords.txt")
#------------------------------------first recommendation-----------------------------------------
#for the first recommendation, group the email by sender, use pandas
df = pd.DataFrame(hamSenderTrain)
df = df[0].value_counts().reset_index()
df.columns = ['sender_describe','sender_values']
x = df['sender_describe'].tolist()
y = df['sender_values'].tolist()

y_array = np.array(y)
y_array = map(lambda value: np.log1p(value), y_array)
index = np.arange(len(y_array))

#bar plot
#parameter setting
bar_width = 0.35
opacity = 0.4
#label setting
plt.xlabel('Email Address')
plt.ylabel('Amount of emails recieved per sender')
plt.title('Amount of emails recieved per sender by email address')
#plot
bar_sender = plt.bar(index, y_array, bar_width, alpha=opacity, color='b')
plt.xticks(range(len(x)), x, rotation='vertical')
plt.show()